# Enhancing Singapore Airlines' Service Through Automated Sentiment Analysis of Customer Reviews



**Motivation**



## Singapore Airlines Customer Reviews Dataset Information

The [Singapore Airlines Customer Reviews Dataset](https://www.kaggle.com/datasets/kanchana1990/singapore-airlines-reviews) aggregates 10,000 anonymized customer reviews, providing a broad perspective on the passenger experience with Singapore Airlines. 

The dimensions are shown below:
- **`published_date`**: Date and time of review publication.
- **`published_platform`**: Platform where the review was posted.
- **`rating`**: Customer satisfaction rating, from 1 (lowest) to 5 (highest).
- **`type`**: Specifies the content as a review.
- **`text`**: Detailed customer feedback.
- **`title`**: Summary of the review.
- **`helpful_votes`**: Number of users finding the review helpful.

## Importing Libraries

Please uncomment the code box below to pip install relevant dependencies for this notebook.

In [1]:
# !pip3 install -r requirements.txt

In [1]:
# Import necessary libraries

# Data manipulation
import pandas as pd
import numpy as np

# Statistical functions
from scipy.stats import zscore

# For concurrency (running functions in parallel)
from concurrent.futures import ThreadPoolExecutor

# For caching (to speed up repeated function calls)
from functools import lru_cache

# For progress tracking
from tqdm import tqdm

# Plotting and Visualisation
import matplotlib.pyplot as plt
import seaborn as sns

# Language Detection packages
# `langdetect` for detecting language
from langdetect import detect as langdetect_detect, DetectorFactory
from langdetect.lang_detect_exception import LangDetectException
# `langid` for an alternative language detection method
from langid import classify as langid_classify

# Text Preprocessing and NLP
# Stopwords (common words to ignore) from NLTK
from nltk.corpus import stopwords

# Tokenizing sentences/words
from nltk.corpus import wordnet

# Tokenizing sentences/words
from nltk.tokenize import word_tokenize
# Lemmatization (converting words to their base form)
from nltk.stem import WordNetLemmatizer
import nltk
# Regular expressions for text pattern matching
import re

# Word Cloud generation
from wordcloud import WordCloud

# For generating n-grams
from nltk.util import ngrams
from collections import Counter

## Data Preparation (Loading CSV)

Load the `singapore_airline_reviews.csv` file into a pandas DataFrame `data`.

# Multinomial NB with Count Vectorizer

Count vectoriser converts a collection of text documents into a matrix of token counts. It simply counts the number of occurrences of each word in the document without considering the importance of frequency of words across the entire corpus.

Count vectoriser counts the raw frequency where each word is weighted equally while tf-idf is weighted by term frequency and rarity across documents.

Count vectoriser is simple and fast as it's just raw counts, while tf-idf requires more computation as it is slightly more complex due to the use of inverse document frequency.

In [2]:
data = pd.read_csv("final_df.csv")

In [3]:
data

,year,month,sentiment,processed_full_review
0,2024,3,Neutral,ok use airlin go singapor london heathrow issu...
1,2024,3,Negative,don give money book paid receiv email confirm ...
2,2024,3,Positive,best airlin world best airlin world seat food ...
3,2024,3,Negative,premium economi seat singapor airlin not worth...
4,2024,3,Negative,imposs get promis refund book flight full mont...
...,...,...,...,...
11513,2021,11,Negative,websit buggi paid first busi class ticket webs...
11514,2021,10,Negative,reduc level qualiti servic fear futur airlin t...
11515,2021,10,Negative,chang would cost usd book ticket singapor airl...
11516,2021,8,Negative,disappoint flight check secur check frankfurt ...


In [4]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, classification_report

vectorizer = CountVectorizer(max_features=1000)
X = vectorizer.fit_transform(data['processed_full_review'])

X_train, X_test, y_train, y_test = train_test_split(X, data['sentiment'],test_size=0.3, random_state=42)

nb_model = MultinomialNB()
nb_model.fit(X_train,y_train)

nb_predictions = nb_model.predict(X_test)

print(accuracy_score(y_test,nb_predictions))
print(classification_report(y_test, nb_predictions))

0.8240740740740741
              precision    recall  f1-score   support

    Negative       0.78      0.69      0.73       692
     Neutral       0.38      0.64      0.48       346
    Positive       0.95      0.89      0.92      2418

    accuracy                           0.82      3456
   macro avg       0.70      0.74      0.71      3456
weighted avg       0.86      0.82      0.84      3456



# RF with Count Vectorizer

In [5]:
from sklearn.ensemble import RandomForestClassifier

rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
rf_model.fit(X_train, y_train)

rf_predictions = rf_model.predict(X_test)
print("Random Forest Accuracy:", accuracy_score(y_test, rf_predictions))
print("Random Forest Classification Report:\n", classification_report(y_test, rf_predictions))

Random Forest Accuracy: 0.8518518518518519
Random Forest Classification Report:
               precision    recall  f1-score   support

    Negative       0.81      0.78      0.79       692
     Neutral       0.88      0.08      0.15       346
    Positive       0.86      0.98      0.92      2418

    accuracy                           0.85      3456
   macro avg       0.85      0.61      0.62      3456
weighted avg       0.85      0.85      0.82      3456



# Log Regression with Count Vectorizer

In [6]:
from sklearn.linear_model import LogisticRegression

clf = LogisticRegression(random_state=42, max_iter=100).fit(X_train, y_train)
clf_predictions = clf.predict(X_test)
print("Accuracy:", accuracy_score(y_test, clf_predictions))
print("Classification Report:\n", classification_report(y_test, clf_predictions))

Accuracy: 0.8449074074074074
Classification Report:
               precision    recall  f1-score   support

    Negative       0.76      0.76      0.76       692
     Neutral       0.43      0.36      0.39       346
    Positive       0.92      0.94      0.93      2418

    accuracy                           0.84      3456
   macro avg       0.70      0.69      0.69      3456
weighted avg       0.84      0.84      0.84      3456



/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


When using count vectoriser, log regression does not converge since count vectoriser produces raw counts for each word, which can result in large values in the feature matrix, especially if a word appears very frequently in a document while TF-IDF normalises these counts by down-weighting common words and dividing by the document length, leading to smaller values in the feature matrix.

Log regression is sensitive to the scale of features. Since Count Vectoriser can produce features of widely varying scales (some words may appear 1000 times, others only once), this can affect convergence. TF-IDF normalises the data, results in features that are more evenly scaled, leading to easier optimisation and quicker convergence.

Fixed by increasing max_iter from 100 to 200 OR scaling data.

In [7]:
from sklearn.linear_model import LogisticRegression

clf = LogisticRegression(random_state=42, max_iter=200).fit(X_train, y_train)
clf_predictions = clf.predict(X_test)
print("Accuracy:", accuracy_score(y_test, clf_predictions))
print("Classification Report:\n", classification_report(y_test, clf_predictions))

Accuracy: 0.84375
Classification Report:
               precision    recall  f1-score   support

    Negative       0.76      0.76      0.76       692
     Neutral       0.43      0.36      0.39       346
    Positive       0.92      0.94      0.93      2418

    accuracy                           0.84      3456
   macro avg       0.70      0.68      0.69      3456
weighted avg       0.84      0.84      0.84      3456



In [8]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler(with_mean=False)  # with_mean=False because sparse matrices don't support centering
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

clf = LogisticRegression(random_state=42).fit(X_train_scaled, y_train)
clf_predictions = clf.predict(X_test_scaled)

print("Accuracy:", accuracy_score(y_test, clf_predictions))
print("Classification Report:\n", classification_report(y_test, clf_predictions))


Accuracy: 0.8035300925925926
Classification Report:
               precision    recall  f1-score   support

    Negative       0.70      0.70      0.70       692
     Neutral       0.34      0.39      0.36       346
    Positive       0.91      0.89      0.90      2418

    accuracy                           0.80      3456
   macro avg       0.65      0.66      0.65      3456
weighted avg       0.81      0.80      0.81      3456

